In [2]:
import os
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_origin
import geopandas as gpd
from shapely.geometry import box, mapping
from tqdm import tqdm
import pandas as pd
from rtree import index

## Making grids to further download the correcponding images for training irrigation (supervised and finetuning)
run only one time

In [7]:
root_dir = "./rice_practices/mask"
output_geojson = "./rice_practices/mask/partial_labels_grids.geojson"

bbox_records = []

# Walk through all .tif files
for dirpath, _, filenames in os.walk(root_dir):
    for fname in filenames:
        if not fname.lower().endswith(".tif"):
            continue

        tif_path = os.path.join(dirpath, fname)
        try:
            with rasterio.open(tif_path) as src:
                bounds = src.bounds
                crs = src.crs
                geom = box(*bounds)
                bbox_records.append({"geometry": geom, "filename": os.path.relpath(tif_path, root_dir)})
        except Exception as e:
            print(f"Failed to read {tif_path}: {e}")

# Create GeoDataFrame
if bbox_records:
    gdf = gpd.GeoDataFrame(bbox_records, crs=crs)
    gdf.to_file(output_geojson, driver="GeoJSON")
    print(f"\nSaved {len(gdf)} raster bboxes to {output_geojson}")
else:
    print("No .tif files found or none were valid.")



Saved 735 raster bboxes to ./rice_practices/mask/partial_labels_grids.geojson


## Make partial labels with unknown fields as -1
### Paths


In [27]:
raster_dir = "/home/airg/rabedi/thesis/mittilabs/rice_practices/mask/2024_11"

In [28]:
# cropfield_layer1 = gpd.read_file("/home/airg/rabedi/thesis/mittilabs/merged_polygons/merged_AP.geojson")
# cropfield_layer2 = gpd.read_file("/home/airg/rabedi/thesis/mittilabs/merged_polygons/merged_TG.geojson")
output_raster_dir = "/home/airg/rabedi/thesis/mittilabs/rice_practices/mask_ready/2024_11"
os.makedirs(output_raster_dir, exist_ok=True)

In [29]:
# target_crs = "EPSG:32644"
# cropfields = gpd.GeoDataFrame(
#     pd.concat([cropfield_layer1, cropfield_layer2], ignore_index=True),
#     crs=cropfield_layer1.crs
# ).to_crs(target_crs)
# print(f"Cropfields reprojected to {target_crs}")

In [30]:
cropfield_index = index.Index()
for i, geom in enumerate(cropfields.geometry):
    cropfield_index.insert(i, geom.bounds)

print("Built spatial index for cropfield polygons.")

# Processing rasters
modified_count = 0

for fname in tqdm(os.listdir(raster_dir)):
    if not fname.endswith(".tif"):
        continue

    raster_path = os.path.join(raster_dir, fname)
    with rasterio.open(raster_path) as src:
        image = src.read(1)
        transform = src.transform
        crs = src.crs
        bounds = src.bounds
        height, width = image.shape
        dtype = image.dtype

        if crs.to_epsg() != 32644:
            raise ValueError(f"Raster {fname} is not in EPSG:32644!")

        chip_box = box(*bounds)

        # Using R-tree to get only possibly intersecting polygons
        candidate_idxs = list(cropfield_index.intersection(chip_box.bounds))
        if not candidate_idxs:
            print(f"No overlapping polygons for {fname}")
            continue

        candidates = cropfields.iloc[candidate_idxs]
        cropfields_clip = candidates[candidates.intersects(chip_box)]

        if cropfields_clip.empty:
            print(f"No real intersection for {fname}")
            continue

        # Rasterizing the clipped polygons
        mask = rasterize(
            [(geom, 1) for geom in cropfields_clip.geometry],
            out_shape=(height, width),
            transform=transform,
            fill=0,
            dtype="uint8"
        ).astype(bool)

        # Only modify pixels that are 0 and inside polygons
        modify_mask = (image == 0) & mask
        if np.any(modify_mask):
            image = image.astype(np.int32) if np.issubdtype(image.dtype, np.uint8) else image
            image[modify_mask] = -1

            out_path = os.path.join(output_raster_dir, fname)
            profile = src.profile
            profile.update(compress='lzw', dtype=image.dtype)

            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(image, 1)

            modified_count += 1
            print(f"Modified and saved: {fname}")
        else:
            print(f"No pixel changed for {fname}. Skipping write.")

print(f"\nDone. Modified and saved {modified_count} raster chips.")


Built spatial index for cropfield polygons.


  1%|▎                                          | 1/154 [00:00<02:03,  1.24it/s]

Modified and saved: raster_11_469879.tif


  1%|▌                                          | 2/154 [00:01<01:23,  1.82it/s]

Modified and saved: raster_11_469921.tif


...

Modified and saved: raster_11_469848.tif


 99%|████████████████████████████████████████▋| 153/154 [01:03<00:00,  2.01it/s]

Modified and saved: raster_11_469899.tif


100%|█████████████████████████████████████████| 154/154 [01:03<00:00,  2.42it/s]

Modified and saved: raster_11_469910.tif

Done. Modified and saved 151 raster chips.
